# Visualizing $G(T,P)$
Written by Dr. Steven Neshyba (University of Puget Sound)

Adapted for Chem 152 at Santa Clara University by Dr. Grace Stokes

## Overview:
Today, we're going to construct $G(T,P)$ for various phases of water. How do we do that? One strategy is by integrating the differential equation of state for the Gibbs energy,

$$
dG = -S dT + V dP \ \ \ \ (1)
$$

That might seem like a lot of work, but we already have a lot of the tools for this: we know how to construct $S(T,P)$ from *its* slopes (see Python 6--*Visualizing $S(T,P)$*), and we have expressions for the volume of water in its various phases (solid, liquid, gas).

## Learning Goals
- Gain familiarity with what the Gibbs energy thermodynamic surface looks like in a temperature-pressure state space
- Recognize the relationship between the *intersection of Gibbs energy* surfaces (liquid and gas) and the *Clausius-Clapeyron* equation. 

In [ ]:
pip install pint

In [ ]:
# Import libraries
import pint; from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
import numpy as np
import matplotlib.pyplot as plt
import PchemLibrary as PL
from mpl_toolkits.mplot3d import axes3d
%matplotlib notebook

### T,P state space
In the cell below, we will make a 3-D grid of temperatures and pressures with the with the following specifications:

- Temperature should run from 273 K to 400 K (and have at least 100 points)
- Pressure should runn from 1000 Pa to 200000 Pa (also at least 100 points)

We will also attach units to each.

In [ ]:
# This code lays out the state space
Tgrid, Pgrid = PL.Statespace([273,400,100],[1000,200000,101])
# Attach units to Tgrid (K) and Pgrid (Pa)
Pgrid = AssignQuantity(Pgrid,'Pa')
Tgrid = AssignQuantity(Tgrid,'K')

### Some general-purpose constants
Values in the cell below should correspond to "standard state" conditions.

Ooops, I made 3 mistakes! Please fix the three values in the cell below...

In [67]:
# Gas constant in SI units
R = AssignQuantity(0.08206,'J /mol/K') #What is value for R in terms of J/mol K?

# Standard state conditions, in SI units
T0 = AssignQuantity(760,'K') #What is standard temperature in Kelvin?
P0 = AssignQuantity(298,'pascal') #How many Pascals are equivalent to 1 bar?

### Constants pertaining to water
In the previous exercise, we looked up constants for three phases of water - See https://webbook.nist.gov/chemistry/name-ser/ for standard-state entropy values. 

In [ ]:
# No need to make changes to this cell. just hit "SHIFT_ENTER"
# Some SI units related to entropy
EUnits = 'J/K/mol'
dSdTUnits = 'J / kelvin^2 /mole'
dSdPUnits = 'meter ** 3 / kelvin / mole'
EnergyUnits = 'J/mol'

# Molar mass of water
M = AssignQuantity(0.018,'kg/mol')

# Parameters for gaseous water
Vgrid_gas = R*Tgrid/Pgrid
CP_gas = AssignQuantity(33.6,EUnits)
SS_gas = [T0,P0,AssignQuantity(188.835,EUnits)] # The standard state entropy


# Parameters for liquid water
Vgrid_liq = AssignQuantity(18e-6,'m^3 /mol')*np.ones(np.shape(Pgrid)) # Molar volume, as a state-space grid
beta_liq = AssignQuantity(2.1e-4,'1/K') # Thermal expansivity
CP_liq = AssignQuantity(75.4,EUnits) # Heat capacity
SS_liq = [T0,P0,AssignQuantity(69.95,EUnits)] # The standard state entropy

## Integrate the differential equation of state for the Gibbs energy from the differential equations for entropy
Below, we use the same state-space integrator we used in *Visualizing $S(T,P)$*) to calculate the entropies of water as a function of pressure and temperature. 
Then, we calculate the $G(T,P)$ thermodynamic surface of water vapor (gas) by integrating $dG = -S_{gas}dT+V_{gas}dP$.


In [ ]:
# No need to make changes to this cell, just hit "SHIFT-ENTER"
# We derived and determined these equations from Python 6
dSdT_gas = CP_gas/Tgrid # dSdT for the gas
dSdP_gas = -R/Pgrid # dSdP for the gas


# Calculate the entropy of the gas as a thermodynamic surface
S_gas = PL.Integrator([Tgrid, Pgrid], dSdT_gas, dSdP_gas, AssignQuantity, Units=EUnits, SState=SS_gas)

# Calculate the Gibbs energy of the gas using standard state Gibbs energy of gas
GS_gas = [T0,P0,AssignQuantity(-228600,'J/mol')] # The standard state Gibbs energy
G_gas = PL.Integrator([Tgrid, Pgrid], -S_gas, Vgrid_gas, AssignQuantity, Units=EnergyUnits, SState=GS_gas)

# Plot the gas' Gibbs Energy
myzlim = [-260000, -220000]
ax = PL.plot_surface(Tgrid, Pgrid, G_gas, color='gray')
ax.set_xlabel(str(Tgrid.units))
ax.set_ylabel(str(Pgrid.units))
ax.set_zlabel(str(G_gas.units))
ax.set_zlim(myzlim)
ax.set_title('G (gas)')

## We will do the same calculations to calculate and plot Gibbs energy of a liquid
We also calculate $G(T,P)$ of liquid water by integrating $dG = -S_{liq}dT+V_{liq}dP$ using the standard-state Gibbs energy of the liquid!

In [ ]:
dSdT_liq = ...  # dSdT for the liquid
dSdP_liq = ...  # dSdP for the liquid
# Calculate the entropy of the liquid as a thermodynamic surface

# Calculate the Gibbs energy of the gas using standard state Gibbs energy of liquid
GS_liq = [T0,P0,AssignQuantity(-237100,EnergyUnits)] # The standard state Gibbs energy

# Plot the liquid's Gibbs Energy in blue

# Overlay Gibbs Energies
To answer the following Pause for Analysis Questions, it may help to plot $G(T,P)$ for liquid and gas on the same figure.


In [ ]:
# Plot gas and liquid together
ax = PL.plot_surface(Tgrid, Pgrid, G_gas, color='gray')
ax = PL.plot_surface(Tgrid, Pgrid, G_liq, color='blue', overlay=True,ax=ax)
ax.set_xlabel(str(Tgrid.units))
ax.set_ylabel(str(Pgrid.units))
ax.set_zlabel(str(G_liq.units))
ax.set_zlim(myzlim)
ax.set_title('G (liq=blue, gas=gray)')

## Pause for analysis #1:
Why is the slope of the curve for G(T,P) for a gas in the temperature direction steeper than for a liquid?

## Pause for analysis #2:
Why is the slope of the curve for G(T,P) for a gas in the pressure direction ALSO steeper than for a liquid?

## Pause for analysis #3:

If we graphed the intersection between the blue and grey graphs, we would see the pressures and temperatures at which the Gibbs energy of the gas equals that of the liquid. This would be the liquid-gas phase coexistence curve in a phase diagram! What equation can we use to model this curve?